# **Part 1: Take a Look at Cartpole Rl Agent**

#### 1. According to the tutorials, if we want to edit the environment configuration, action space, observation space, reward function, or termination condition of the Isaac-Cartpole-v0 task, which file should we look at, and where is each part located?

**Source code:** https://github.com/isaac-sim/IsaacLab/blob/main/source/extensions/omni.isaac.lab_tasks/omni/isaac/lab_tasks/manager_based/classic/cartpole/cartpole_env_cfg.py

We should look at cartpole_env_cfg.py that located in Home/IsaacLab/blob/main/source/extensions/omni.isaac.lab_tasks/omni/isaac/lab_tasks/manager_based/classic/cartpole/cartpole_env_cfg.py and each part located

**Environment configuration** in line 157-181


In [ ]:
@configclass
class CartpoleEnvCfg(ManagerBasedRLEnvCfg):
    """Configuration for the cartpole environment."""

    # Scene settings
    scene: CartpoleSceneCfg = CartpoleSceneCfg(num_envs=4096, env_spacing=4.0)
    # Basic settings
    observations: ObservationsCfg = ObservationsCfg()
    actions: ActionsCfg = ActionsCfg()
    events: EventCfg = EventCfg()
    # MDP settings
    rewards: RewardsCfg = RewardsCfg()
    terminations: TerminationsCfg = TerminationsCfg()

    # Post initialization
    def __post_init__(self) -> None:
        """Post initialization."""
        # general settings
        self.decimation = 2
        self.episode_length_s = 5
        # viewer settings
        self.viewer.eye = (8.0, 0.0, 5.0)
        # simulation settings
        self.sim.dt = 1 / 120
        self.sim.render_interval = self.decimation

**Action space** in line 58-62

In [ ]:
@configclass
class ActionsCfg:
    """Action specifications for the MDP."""

    joint_effort = mdp.JointEffortActionCfg(asset_name="robot", joint_names=["slider_to_cart"], scale=100.0)

**Observation space** in line 65-85

In [ ]:
@configclass
class ObservationsCfg:
    """Observation specifications for the MDP."""

    @configclass
    class PolicyCfg(ObsGroup):
        """Observations for policy group."""

        # observation terms (order preserved)
        joint_pos_rel = ObsTerm(func=mdp.joint_pos_rel)
        joint_vel_rel = ObsTerm(func=mdp.joint_vel_rel)

        def __post_init__(self) -> None:
            self.enable_corruption = False
            self.concatenate_terms = True

    # observation groups
    policy: PolicyCfg = PolicyCfg()

**Reward function** in line 111-136

In [ ]:
@configclass
class RewardsCfg:
    """Reward terms for the MDP."""

    # (1) Constant running reward
    alive = RewTerm(func=mdp.is_alive, weight=1.0)
    # (2) Failure penalty
    terminating = RewTerm(func=mdp.is_terminated, weight=-2.0)
    # (3) Primary task: keep pole upright
    pole_pos = RewTerm(
        func=mdp.joint_pos_target_l2,
        weight=-1.0,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["cart_to_pole"]), "target": 0.0},
    )
    # (4) Shaping tasks: lower cart velocity
    cart_vel = RewTerm(
        func=mdp.joint_vel_l1,
        weight=-0.01,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["slider_to_cart"])},
    )
    # (5) Shaping tasks: lower pole angular velocity
    pole_vel = RewTerm(
        func=mdp.joint_vel_l1,
        weight=-0.005,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["cart_to_pole"])},
    )



**Termination condition** in line 139-149

In [ ]:
@configclass
class TerminationsCfg:
    """Termination terms for the MDP."""

    # (1) Time out
    time_out = DoneTerm(func=mdp.time_out, time_out=True)
    # (2) Cart out of bounds
    cart_out_of_bounds = DoneTerm(
        func=mdp.joint_pos_out_of_manual_limit,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["slider_to_cart"]), "bounds": (-3.0, 3.0)},
    )


#### 2. What are the action space and observation space for an agent defined in the Isaac-Cartpole-v0 task?

Action space is action controls the torque applied to the joint which moves the cart horizontally.

In [ ]:
@configclass
class ActionsCfg:
    """Action specifications for the MDP."""

    joint_effort = mdp.JointEffortActionCfg(asset_name="robot", joint_names=["slider_to_cart"], scale=100.0)

Observation space includes:
1. Relative Position (joint_pos_rel):
- Cart Position: Horizontal position of the cart.
- Pole Angle: Angular position of the pole relative to the vertical.
2. Relative Velocity (joint_vel_rel):
- Cart Velocity: Horizontal velocity of the cart.
- Pole Angular Velocity: Angular velocity of the pole.

In [ ]:
@configclass
class ObservationsCfg:
    """Observation specifications for the MDP."""

    @configclass
    class PolicyCfg(ObsGroup):
        """Observations for policy group."""

        # observation terms (order preserved)
        joint_pos_rel = ObsTerm(func=mdp.joint_pos_rel)
        joint_vel_rel = ObsTerm(func=mdp.joint_vel_rel)

        def __post_init__(self) -> None:
            self.enable_corruption = False
            self.concatenate_terms = True

    # observation groups
    policy: PolicyCfg = PolicyCfg()

#### 3. How can episodes in the Isaac-Cartpole-v0 task be terminated?

In the Isaac-Cartpole-v0 task, episodes can be terminated in two ways as

1. Time out: The episode terminates when a predefined time limit is reached.

In [ ]:
time_out = DoneTerm(func=mdp.time_out, time_out=True)

2. Cart out of bounds: The episode terminates if the cart moves beyond specified position limits, which checks whether the cart's position lies outside the bounds (-3.0, 3.0)


In [ ]:
cart_out_of_bounds = DoneTerm(
        func=mdp.joint_pos_out_of_manual_limit,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["slider_to_cart"]), "bounds": (-3.0, 3.0)},
    )


#### 4. How many reward terms are used to train an agent in the Isaac-Cartpole-v0 task?

There are 5 reward terms line 111-136

1. Constant running reward เป็น Reward (+1) ที่ให้เมื่อ Agent ยังสามารถมีชีวิตรอดได้ในแต่ละ Step เพื่อสนับสนุนให้ Agent พยายามอยู่รอดในเกมให้ได้นานที่สุด


In [ ]:
alive = RewTerm(func=mdp.is_alive, weight=1.0)

2. Failure penalty เป็น Reward ลดแต้ม (-2) เมื่อ Cartpole ออกนอกขอบเขต เพื่อกระตุ้นให้ Agent หลีกเลี่ยงการเข้าสู่สถานะล้มเหลว

In [ ]:
 terminating = RewTerm(func=mdp.is_terminated, weight=-2.0)

3. Primary task: keep pole upright เป็นการกระตุ้นให้เสาตั้งตรง หากเสามีการเอียงจากตำแหน่งตั้งตรง (Target Position) จะได้บทลงโทษ เป็น Reward ติดลบมากขึ้นตามระยะห่างระหว่าง Pole Position กับ Target Position


In [ ]:
pole_pos = RewTerm(
        func=mdp.joint_pos_target_l2,
        weight=-1.0,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["cart_to_pole"]), "target": 0.0},
    )

4. Shaping tasks: lower cart velocity เป็น Reward ที่สนับสนุนให้ Agent ลด Cart Velocity เพื่อเพิ่มความเสถียรให้กับระบบ ให้ Agent เรียนรู้ที่จะเคลื่อนที่ด้วยความเร็วที่เหมาะสม

In [ ]:
cart_vel = RewTerm(
        func=mdp.joint_vel_l1,
        weight=-0.01,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["slider_to_cart"])},
    )

5. Shaping tasks: lower pole angular velocity เป็น Reward ที่ลด Pole Angular Velocity เพื่อเพิ่มความเสถียรให้กับระบบ ป้องกันไม่ให้เสาแกว่งมากเกินไปจนเสียสมดุล

In [ ]:
    pole_vel = RewTerm(
        func=mdp.joint_vel_l1,
        weight=-0.005,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["cart_to_pole"])},
    )


# **Part 2: Playing with Cartpole Rl Agent**

Let us adjust the weight of each reward term specified in the Isaac-Cartpole-v0 task and train the agent. Which results will be affected by this adjustment, and why? Submit the answers.

You may further explore by modifying other aspects, such as the agent's action space, observation space, and termination conditions.

### Experiment 0
กำหนด reward ตามค่า Default เดิมใน IsaacLab คือ
1. Constant running reward เป็น Reward (+1) ที่ให้เมื่อ Agent ยังสามารถมีชีวิตรอดได้ในแต่ละ Step เพื่อสนับสนุนให้ Agent 
2. Failure penalty เป็น Reward ลดแต้ม (-2) เมื่อ Cartpole ออกนอกขอบเขต เพื่อกระตุ้นให้ Agent หลีกเลี่ยงการเข้าสู่สถานะล้มเหลว
3. Primary task: keep pole upright เป็นการกระตุ้นให้เสาตั้งตรง หากเสามีการเอียงจากตำแหน่งตั้งตรง (Target Position) จะได้บทลงโทษ เป็น Reward (-1) ติดลบมากขึ้นตามระยะห่างระหว่าง Pole Position กับ Target Position
4. Shaping tasks: lower cart velocity เป็น Reward (-0.01) ที่สนับสนุนให้ Agent ลด Cart Velocity เพื่อเพิ่มความเสถียรให้กับระบบ ให้ Agent เรียนรู้ที่จะเคลื่อนที่ด้วยความเร็วที่เหมาะสม
5. Shaping tasks: lower pole angular velocity เป็น Reward (-0.005) ที่ลด Pole Angular Velocity เพื่อเพิ่มความเสถียรให้กับระบบ ป้องกันไม่ให้เสาแกว่งมากเกินไปจนเสียสมดุล

#### Last Result
| **Metric**            | **Value**         | **Meaning** |
|----------------------|------------------|--------------------------------|
| **Episode Length (`ep_len_mean`)** | **286** | ค่าเฉลี่ยของ time step ที่ agent สามารถมีชีวิตรอดใน environment ค่าที่มากขึ้นหมายถึง agent สามารถ balance pole ได้ดีขึ้น |
| **Episode Reward (`ep_rew_mean`)** | **4.7** | ค่าผลรวมของ reward ที่ agent ได้รับในแต่ละ Episode ค่าที่สูงขึ้นบ่งบอกว่าการกระทำของ agent ได้รับรางวัลที่ดีขึ้น |
| **Entropy Loss (`entropy_loss`)** | **-0.163** | ค่า entropy loss ที่มากขึ้นหมายถึง agent มี policy ในการ exploration มากขึ้น ในขณะที่ค่าที่ลดลงหมายถึง agent กำลัง converging ไปสู่ policy ที่แน่นอนมากขึ้น |
| **Explained Variance (`explained_variance`)** | **0.873** | บ่งบอกว่า value function สามารถพยากรณ์ future reward ได้ดีแค่ไหน ค่าที่สูงขึ้นหมายถึงการเรียนรู้ที่ stable และ value function มีความแม่นยำมากขึ้น |
| **Policy Gradient Loss (`policy_gradient_loss`)** | **-0.00446** | วัดอัตราการเปลี่ยนแปลงของ policy ค่าที่ต่ำลงบ่งบอกว่า agent ปรับ policy น้อยลง แสดงถึงการเรียนรู้ที่เริ่มนิ่งขึ้น |
| **Standard Deviation (`std`)** | **0.286** | วัดความหลากหลายของ action ที่ agent เลือก ค่าที่ต่ำลงหมายถึง agent มีความมั่นใจในการเลือก action มากขึ้น |
| **Value Loss (`value_loss`)** | **0.00281** | วัดความผิดพลาดของ value function ในการพยากรณ์ future rewards ค่าที่สูงขึ้นอาจหมายถึง agent ยังมีความไม่แน่นอนในการเรียนรู้ (higher exploration) ในขณะที่ค่าที่ต่ำลงแสดงถึงความมั่นใจของ model ที่สูงขึ้น |

### Experiment 1
เพิ่มค่า Alive Reward จาก +1.0 เป็น +2.0

#### Last Result

| **Metric**            | **Default Value** | **Value**         | **Interpretation** |
|----------------------|------------------|------------------|--------------------------------|
| **Episode Length (`ep_len_mean`)** | **286** | **297** | agent พยายามที่จะมีชีวิตรอดใน environment ได้นานขึ้น เนื่องจาก Alive Reward กระตุ้นให้เน้นการเอาชีวิตรอด |
| **Episode Reward (`ep_rew_mean`)** | **4.7** | **9.84** | ค่าผลรวมของ reward สูงขึ้นเนื่องจาก Alive Reward เพิ่มขึ้น 2 เท่า ทำให้ทุก timestep มีผลต่อ reward มากขึ้น |
| **Entropy Loss (`entropy_loss`)** | **-0.163** | **-0.138** | ค่า entropy loss ที่ลดลงเล็กน้อย แสดงว่า agent เริ่ม converging สู่ policy ที่ deterministic มากขึ้น |
| **Explained Variance (`explained_variance`)** | **0.873** | **0.506** | ลดลงเพราะ agent มุ่งเน้นแค่การอยู่รอด มากกว่าการ optimize future rewards หรือ balancing อื่นๆ |
| **Standard Deviation (`std`)** | **0.286** | **0.278** | agent มีความมั่นใจใน action มากขึ้น แต่ไม่ได้ต่างจากเดิมมาก |
| **Value Loss (`value_loss`)** | **0.00281** | **0.000373** | ลดลงเพราะ agent สามารถประมาณค่าผลตอบแทนสะสมได้ดีขึ้น และ policy มีความเสถียรมากขึ้น |


### Experiment 2
ลดค่า Alive Reward จาก +1.0 เป็น +0.5

#### Last Result

| **Metric**            | **Default Value** | **Reduced Alive Reward Value** | **Comparison** |
|----------------------|------------------|-------------------------------|--------------------------------|
| **Episode Length (`ep_len_mean`)** | **286** | **290** | เพิ่มขึ้นเล็กน้อย เพราะ agent ยังสามารถ balance ได้ในระดับใกล้เคียงเดิม เนื่องจาก Alive Reward ที่ลดลงไม่ได้มีผลกระทบมาก |
| **Episode Reward (`ep_rew_mean`)** | **4.7** | **2.35** | ลดลงชัดเจน เนื่องจาก Alive Reward ลดลงครึ่งหนึ่ง ส่งผลต่อรางวัลรวมโดยตรง |
| **Entropy Loss (`entropy_loss`)** | **-0.163** | **0.302** | เพิ่มขึ้นอย่างมาก แสดงว่า agent มีการ exploration มากขึ้น เพราะแรงจูงใจในการเอาชีวิตรอดลดลง ทำให้สามารถลอง action อื่นได้มากขึ้น |
| **Explained Variance (`explained_variance`)** | **0.873** | **0.764** | ลดลงเล็กน้อย แสดงว่า value function มีความแม่นยำน้อยลงเล็กน้อยในการพยากรณ์ future rewards |
| **Standard Deviation (`std`)** | **0.286** | **0.179** | ลดลง แสดงว่า agent มี policy ที่ deterministic มากขึ้น แต่ลด diversity ของ action ที่เลือก |
| **Value Loss (`value_loss`)** | **0.00281** | **3.53e-05** | ลดลงอย่างมาก แสดงว่า value function มีความมั่นใจมากขึ้นในค่า reward ที่ predict|

### Experiment 3
เพิ่มการลงโทษ เมื่อเกิดการ Terminate (เมื่อ Cartpole ออกนอกขอบเขต)
ด้วยการลดค่า Termination Reward ลงจาก -2.0 เป็น -5.0

#### Last Result

| **Metric**            | **Default Value** | **Reduced Termination Reward Value** | **Comparison** |
|----------------------|------------------|-------------------------------|--------------------------------|
| **Episode Length (`ep_len_mean`)** | **286** | **300** | เพิ่มขึ้นเพราะ agent พยายามหลีกเลี่ยงการ terminate เนื่องจากการลงโทษที่สูงขึ้น |
| **Episode Reward (`ep_rew_mean`)** | **4.7** | **4.93** | ใกล้เคียงค่า Default แสดงว่า agent ให้ความสำคัญกับการหลีกเลี่ยง termination reward แต่ไม่ได้เพิ่มพฤติกรรมที่ส่งผลต่อ reward อื่น |
| **Entropy Loss (`entropy_loss`)** | **-0.163** | **-0.244** | ลดลง แสดงว่า agent มี policy ที่ deterministic มากขึ้น เนื่องจากต้องหลีกเลี่ยงความเสี่ยงจาก termination |
| **Explained Variance (`explained_variance`)** | **0.873** | **0.95** | เพิ่มขึ้นเล็กน้อย บ่งบอกว่า value function predict ได้แม่นยำขึ้นในสถานการณ์ที่หลีกเลี่ยงการ terminate |
| **Standard Deviation (`std`)** | **0.286** | **0.308** | ค่าที่เพิ่มขึ้นแสดงว่า agent มี ความหลากหลายของ action เพิ่มขึ้นเล็กน้อย|
| **Value Loss (`value_loss`)** | **0.00281** | **1.63e-05** | ลดลงอย่างมาก แสดงว่า value function มีความมั่นใจสูงขึ้นในการ predict future rewards |

### Experiment 4
ลดการลงโทษ เมื่อเกิดการ Terminate (เมื่อ Cartpole ออกนอกขอบเขต)
ด้วยการเพิ่มค่า Termination Reward จาก -2.0 เป็น -1.0

#### Last Result

| **Metric**            | **Default Value** | **Reduced Termination Penalty Value** | **Comparison** |
|----------------------|------------------|-------------------------------|--------------------------------|
| **Episode Length (`ep_len_mean`)** | **286** | **297** | เพิ่มขึ้นเล็กน้อย เพราะ agent สามารถ balance ได้ในระดับใกล้เคียงเดิม และ termination reward ที่ลดลง ลดความกดดันของ Agent ในการหลีกเลี่ยง termination |
| **Episode Reward (`ep_rew_mean`)** | **4.7** | **4.86** | เพิ่มขึ้นเล็กน้อย เพราะการลงโทษเมื่อ terminate ลดลง ทำให้ reward โดยรวมสูงขึ้น |
| **Entropy Loss (`entropy_loss`)** | **-0.163** | **-0.48** |  ค่า entropy loss ที่ ลดลง แสดงว่า agent กำลัง converging สู่ policy ที่ deterministic มากขึ้น |
| **Explained Variance (`explained_variance`)** | **0.873** | **0.247** | ลดลงอย่างมาก แสดงว่า value function มีความแม่นยำน้อยลงในการพยากรณ์ future rewards เนื่องจาก policy เปลี่ยนแปลงและมี exploration ลดลง |
| **Standard Deviation (`std`)** | **0.286** | **0.39** | เพิ่มขึ้น แสดงว่า agent เลือกทำ action ที่หลากหลายมากขึ้น ซึ่งเป็นผลจากการที่ termination penalty ลดลง ทำให้ลอง action อื่น ๆ ได้มากขึ้น |
| **Value Loss (`value_loss`)** | **0.00281** | **4.08e-05** | ลดลงอย่างมาก แสดงว่า value function มีความมั่นใจมากขึ้นในการพยากรณ์ future rewards |

### Experiment 5 
เพิ่มการลงโทษ เมื่อ pole position อยู่ห่างจากตำแหน่งตั้งตรง
ด้วยกาลดค่า Pole_pos Reward จาก -1.0 เป็น -2.0

#### Last Result

| **Metric**            | **Default Value** | **Increased Pole Position Penalty** | **Comparison** |
|----------------------|------------------|-------------------------------|--------------------------------|
| **Episode Length (`ep_len_mean`)** | **286** | **295** | เพิ่มขึ้นเล็กน้อย แสดงว่า agent พยายามทำให้ pole อยู่ในลักษณะตั้งตรงมากขึ้น เพื่อลดผลกระทบจาก pole position reward ซึ่งช่วยให้ balance ได้นานขึ้น |
| **Episode Reward (`ep_rew_mean`)** | **4.7** | **4.82** | เพิ่มขึ้นเล็กน้อย แสดงว่า agent สามารถหลีกเลี่ยงการได้รับบทลงโทษที่มากขึ้นจาก pole position ได้บ้าง |
| **Entropy Loss (`entropy_loss`)** | **-0.163** | **-0.471** | ลดลงมาก แสดงว่า agent ลดการ exploration เพราะต้องโฟกัสที่การตั้งตรงให้เร็วและมั่นคงยิ่งขึ้น |
| **Explained Variance (`explained_variance`)** | **0.873** | **0.86** | ลดลงเล็กน้อย แสดงว่า value function มีความแม่นยำน้อยลงเล็กน้อยในการพยากรณ์ future rewards อาจเพราะ focus ในการตั้งตรงมากกว่าการ optimize อื่น |
| **Standard Deviation (`std`)** | **0.286** | **0.387** | ค่าที่เพิ่มขึ้นแสดงว่า agent อาจมีความไม่แน่นอนเล็กน้อยในการเลือก action หรือปรับตัวในสถานการณ์ใหม่ |
| **Value Loss (`value_loss`)** | **0.00281** | **0.00089** | ลดลงแสดงว่า value function มีความมั่นใจมากขึ้นในค่า reward ที่ predict มา |

# **Part 3: Mapping RL Fundamentals**

#### 1. What is reinforcement learning and its components according to your understanding? Giving examples of each component according to the diagram consider the Cartpole problem.

Reinforcement Learning คือ แนวทางการเรียนรู้โดยที่ Agent ไม่รู้ optimal sequence of actions มาก่อน Agent จะเรียนรู้ผ่านการลองทำ Action ต่างๆ แล้วรับ Reward เป็น Feedback เพื่อให้เกิดการเรียนรู้ว่าทำ Action ไหนดีหรือไม่ดี โดย Agent จะเรียนรู้เพื่อสร้าง policy ว่าควร Action อย่างไรต่อสถานการณ์ต่างๆ เพื่อให้ได้ Reward มากที่สุด และเพื่อเพิ่ม Value Function ให้ได้มากที่สุด 

**1. Agent:** ตัวแทนที่จะคอยเรียนรู้ผ่าน Reward และสร้าง Policy เพื่อตัดสินใจ ดูว่าใน State นี้ ควรจะ Action อย่างไร
- Cartpole problem: คือสมองของ Cartpole ที่คอยทำหน้าที่ตัดสินใจว่าจาก State (Pole Position) ในปัจจุบัน ควรจะทำ Action อะไร (เคลื่อน Cart ไปทางไหน)

**2. Environment:** ระบบที่ Agent ต้องมีการตอบโต้ด้วย
- Cartpole problem: การจำลองฟิสิกส์สำหรับการเคลื่อนที่ของ Cartpole

**3. State:** สถานการณ์ในเวลาหนึ่ง
- Cartpole problem: Cart Position, Pole Angle, Cart Velocity, Pole Angular Velocity

**4. Reward:** สิ่งที่ใช้ในการประเมินผลลัพธ์ที่จะเกิดจากการกระทำของ Agent ว่าดีหรือไม่ดี เพื่อบอกว่า Agent ควรจะเลือกทำ Action ไหน
- Cartpole problem:
    - Reward +1 เมื่อ Agent ยังมีชีวิตรอดอยู่
    - Reward -2 เมื่อ Cartpole ออกนอกขอบเขตแล้ว Agent เข้าสู่สถานะล้มเหลว
    - โดนลด Reward ตามระยะห่างระหว่าง Pole Position กับ Target Position
    - โดนลด Reward หาก Cart Velocity ไม่เหมาะสม
    - โดนลด Reward หาก Pole Angular Velocity ไม่เหมาะสม

**5. Action:** การกระทำจากการตัดสินใจของ Agent
- Cartpole problem: Action ที่ Agent ตัดสินใจทำ ว่าต้องการเคลื่อน Cart ไปทางไหน ด้วยการกำหนด Torque

**6. Policy:** หลักการที่ Agent ใช้ในการตัดสินใจหลังจากวิเคราะห์ความเป็นไปได้ที่เกิดจากการได้รับ Reward ที่เรียนรู้จากการทำ Action ก่อนหน้า
- Cartpole problem: ฟังก์ชันที่มี Input คือ State ในปัจจุบัน Cart Position, Pole Angle, Cart Velocity, Pole Angular Velocity แล้ว Output ออกมาเป็น Action ในรูปแบบ Torque เพื่อเคลื่อน Cart (เคลื่อนที่ซ้ายหรือขวา)

**7. Value Function:** การประเมิน Reward สะสมที่คาดหวังของ State ปัจจุบัน หลังจากทำตาม Policy
- Cartpole problem: ประเมินรางวัลสะสม (Expected Return) ที่เอเจนต์จะได้รับในอนาคต โดยเริ่มจาก State ปัจจุบันและทำ Action ตาม Policy เช่น หาก State ใดอยู่ใกล้ Goal Point (ที่ Pole ตั้งตรง) มากกว่า จะมีค่า Value Function สูงกว่า เอเจนต์จึงพยายามเคลื่อนที่ไปยัง State นั้น

#### 2. What is the difference between the reward, return, and the value function?

**Reward:** ผลตอบแทนที่ได้รับทันทีหลังจากทำ Action ใน State ปัจจุบัน เป็น Scalar feedback signal ที่สะท้อนถึงคุณภาพของ Action ที่เลือกทำใน timestep นั้นๆ

**Return:** เป็นค่า Predict ผลตอบแทนสะสมในอนาคต (cumulative reward) ที่เริ่มต้นจากเวลาปัจจุบัน(t) ใช้เพื่อช่วยให้ Agent เข้าใจว่า Action ใดจะนำไปสู่รางวัลสะสมสูงสุดในระยะยาว

**Value Function:** การคาดการณ์ Return เฉลี่ยในอนาคตของ State s ที่สนใจ โดยขึ้นอยู่กับ Policy ที่ใช้อยู่ เช่นหาก state ใดให้ค่า Value สูง Agent ก็จะพยายามไปยัง state นั้น

#### 3. Consider policy, state, value function, and model as mathematical functions, what would each one take as input and output?

**Policy** เป็นฟังก์ชันที่กำหนด Action ที่ Agent ควรทำใน State ปัจจุบัน
- input: State ปัจจุบัน และ Action ที่เป็นไปได้ (ไม่จำเป็นสำหรับทุกกรณี)
- output: 
    - ในกรณี Deterministic policy
    ให้ Output เป็น Action เดียวที่เหมาะสมที่สุด
    $\pi(S) = A$
    - ในกรณี Stochastic policy
    ให้ Output เป็นความน่าจะเป็นของแต่ละ Action
    $\pi(A \mid S) = p(A \mid S)$

**State** 

Transition Function $S_{t+1} = u(S_t, At, R{t+1}, O_{t+1})$

- Input:
    - $s_t$: State ปัจจุบัน
    - $a_t$: Action ที่เอเจนต์เลือกใน Timestep ปัจจุบัน
    - $r_{t+1}$: Reward ที่ได้รับหลังทำ $a_t$
    - $o_{t+1}$: Observation ใหม่ที่ได้รับจาก Environment

- Output:
    - $s_{t+1}$: State ใหม่ของระบบใน Timestep ถัดไป

**Value Function**

$v_\pi(s) = \mathbb{E}[G_t \mid S_t = s, A_t \sim \pi(s)]$

- input: State s ที่สนใจ 
- output: ค่าคาดหวังของ Reture (Cumulative Reward) เมื่อเริ่มต้นจาก State s และปฏิบัติตาม Policy $\pi$

**Model** เป็นเครื่องมือในการ Predict ว่าอะไรจะเกิดขึ้นถัดไป เช่นถ้าเป็น model สำหรับ Predict Next Reward จะมี Input และ Output คือ

$R(s, a) = \mathbb{E}[R_{t+1} \mid S_t = s, A_t = a]$

- input: State s ที่สนใจ และ Action a ที่จะเลือกทำ
- output: ประมาณ Reward ที่จะได้รับเมื่อทำ Action a ที่ State s 